In [ ]:
import os
import torch

from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
from torchvision import tv_tensors
from torchvision.transforms.v2 import functional as F
from google.colab import drive
from PIL import Image
import matplotlib.pyplot as plt
import sys
import pandas as pd
from collections import defaultdict
import json
import random
from torch.utils.data import Dataset
import numpy as np
import torchvision.models.detection as detection
from torch.utils.data import DataLoader
import torch.utils
import torch
from PIL import Image as read_image
from torchvision.io import read_image
from torchvision.transforms import v2
from transforms import RandomPhotometricDistort, RandomZoomOut, RandomIoUCrop, RandomHorizontalFlip,RandomVerticalFlip ,ToTensor, Compose
from torchvision.transforms.functional import convert_image_dtype

In [ ]:
drive.mount('/content/drive')
path_base = '/content/drive/MyDrive/Academico/TD8 personal '
sys.path.append(path_base)

Mounted at /content/drive


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:

train_file = path_base + '/classificationDownload (1)/train.csv'
test_file = path_base + '/classificationDownload (1)/test.csv'
classification_file = path_base + '/classificationDownload (1)/classifications.csv'

train_images = pd.read_csv(train_file)['image_filename'].tolist()
test_images = pd.read_csv(test_file)['image_filename'].tolist()


classification_data = pd.read_csv(classification_file)


train_data = classification_data[classification_data['image_filename'].isin(train_images)]
test_data = classification_data[classification_data['image_filename'].isin(test_images)]



def get_transform(train=True):
    transforms = []
    transforms.append(v2.ToDtype(torch.float, scale=True))
    return Compose(transforms)




def group_detections(data):
    grouped_data = {}
    for _, row in data.iterrows():
        image_name = row['image_filename']
        if image_name not in grouped_data:
            grouped_data[image_name] = {
                'image_filename': image_name,
                'image_id': row['image_id'],
                'image_doi': row['image_doi'],
                'classifications': []
            }

        grouped_data[image_name]['classifications'].append({
            'nucleus_x': row['nucleus_x'],
            'nucleus_y': row['nucleus_y'],
            'bethesda_system': row['bethesda_system'],
            'cell_id': row['cell_id']
        })
    return list(grouped_data.values())


train_data_grouped = group_detections(train_data)
test_data_grouped = group_detections(test_data)


train_counts = defaultdict(int)
test_counts = defaultdict(int)


for item in train_data_grouped:
    for classification in item['classifications']:
        train_counts[classification['bethesda_system']] += 1


for item in test_data_grouped:
    for classification in item['classifications']:
        test_counts[classification['bethesda_system']] += 1


class PAPDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

        self.label_map = {
            "SCC": 1,
            "HSIL": 2,
            "LSIL": 3,
            "ASC-H": 4,
            "ASC-US": 5,
            "Negative for intraepithelial lesion": 6
        }

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]

        img = read_image(path_base + '/Imagenes/' + item['image_filename'])

        image_doi = item['image_doi']
        image_id = item['image_id']
        boxes = []
        labels = []
        for classification in item['classifications']:
            nucleus_x = classification['nucleus_x']
            nucleus_y = classification['nucleus_y']
            x_min = nucleus_x - 50
            y_min = nucleus_y - 50
            x_max = nucleus_x + 50
            y_max = nucleus_y + 50
            boxes.append([x_min, y_min, x_max, y_max])
            labels.append(self.label_map[classification['bethesda_system']])


        boxes = torch.as_tensor(boxes, dtype=torch.float32)

        labels = torch.as_tensor(labels, dtype=torch.int64)
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])

        target = {
            "boxes": boxes,
            "labels": labels,
            "area": area,
            "image_id": image_id
        }

        num_objs = boxes.shape[0]
        target["iscrowd"] = torch.zeros((num_objs,), dtype=torch.int64)


        if self.transform:
            img, target = self.transform(img, target)
        return img, target


train_dataset = PAPDataset(train_data_grouped, transform=get_transform(train=True))
test_dataset = PAPDataset(test_data_grouped, transform=get_transform(train=False))

print("Training Counts:", dict(train_counts))
print("Test Counts:", dict(test_counts))
print(f"Training Dataset Size: {len(train_dataset)}")
print(f"Test Dataset Size: {len(test_dataset)}")


Training Counts: {'SCC': 145, 'Negative for intraepithelial lesion': 6043, 'LSIL': 1221, 'HSIL': 1588, 'ASC-H': 808, 'ASC-US': 550}
Test Counts: {'ASC-H': 117, 'Negative for intraepithelial lesion': 736, 'ASC-US': 56, 'LSIL': 139, 'SCC': 16, 'HSIL': 115}
Training Dataset Size: 360
Test Dataset Size: 40


In [ ]:
from ssd import SSDFeatureExtractorResNet_Upgraded, ssd512_resnet50_Upgraded
model = ssd512_resnet50_Upgraded(pretrained = False, load = False)
model = model.to(device)



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 180MB/s]


In [ ]:
from ssd import SSDClassificationHead, SSDRegressionHead
num_classes = 6+1


in_channels = [8192, 3584, 3584, 3328, 256, 256]
num_anchors = [4, 6, 6, 6, 4, 4]
model.head.classification_head = SSDClassificationHead(in_channels, num_anchors, num_classes)


In [ ]:

def collate_fn(batch):
    return tuple(zip(*batch))



batch_size = 16

train_sampler = torch.utils.data.RandomSampler(train_dataset)
test_sampler = torch.utils.data.SequentialSampler(test_dataset)

train_batch_sampler = train_batch_sampler = torch.utils.data.BatchSampler(
            train_sampler, batch_size, drop_last=True)


data_loader = torch.utils.data.DataLoader(
        train_dataset, batch_sampler=train_batch_sampler,
        collate_fn=collate_fn)


data_loader_test = torch.utils.data.DataLoader(
        test_dataset, batch_size=1,
        sampler=test_sampler,
        collate_fn=collate_fn)

In [ ]:
import math
from utils import MetricLogger, warmup_lr_scheduler, SmoothedValue
def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq):
    model.train()
    metric_logger = MetricLogger(delimiter="  ")
    metric_logger.add_meter('lr', SmoothedValue(window_size=1, fmt='{value:.6f}'))
    header = 'Epoch: [{}]'.format(epoch)

    lr_scheduler = None
    if epoch == 0:

        warmup_factor = 1. / 1000
        warmup_iters = min(1000, len(data_loader) - 1)

        lr_scheduler = warmup_lr_scheduler(optimizer, warmup_iters, warmup_factor)

    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) if not isinstance(v, int) else v for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)


        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        print(loss_value)

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            break

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

    return metric_logger



'\nMetricLogger te ayuda a monitorear métricas a lo largo del entrenamiento.\nWarmup Learning Rate Scheduler te permite empezar el entrenamiento de manera más suave ajustando gradualmente la tasa de aprendizaje durante las primeras iteraciones para evitar inestabilidad.\n'

In [ ]:
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_utils.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/coco_eval.py
!wget https://raw.githubusercontent.com/pytorch/vision/main/references/detection/transforms.py
!pip install -U pycocotools

--2024-11-20 20:53:04--  https://raw.githubusercontent.com/pytorch/vision/main/references/detection/engine.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4063 (4.0K) [text/plain]
Saving to: ‘engine.py’

engine.py           100%[===================>]   3.97K  --.-KB/s    in 0s      

2024-11-20 20:53:05 (87.6 MB/s) - ‘engine.py’ saved [4063/4063]

--2024-11-20 20:53:05--  https://raw.githubusercontent.com/pytorch/vision/main/references/detection/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8388 (8.2K) [text/plain]
S

In [ ]:
from torchvision.datasets import VOCDetection
from torchvision.transforms import Compose, ToTensor, RandomHorizontalFlip, Normalize
from engine import evaluate
import time
import datetime


params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(
        params, lr = 0.003, momentum=0.9, weight_decay = 0.0005)

lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[80,110], gamma=0.1)
model.to(device)
best_ap = 0.0

print("Start training")
start_time = time.time()
for epoch in range(50):
  train_one_epoch(model, optimizer, data_loader, device, epoch, 20)
  lr_scheduler.step()
  ap = evaluate(model, data_loader_test, device=device).coco_eval["bbox"].stats[0]
  if ap > best_ap:
    best_ap = ap
    best_model_weights = model.state_dict()

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))








Start training
24.387863159179688
Epoch: [0]  [ 0/22]  eta: 0:04:24  lr: 0.000146  time: 12.0329  data: 8.7954  max mem: 11006
23.843324661254883
21.170866012573242
16.197185516357422
13.77003288269043
12.812237739562988
11.776643753051758
9.628870010375977
10.875503540039062
8.046767234802246
8.250761032104492
7.50226354598999
7.999004364013672
7.740845680236816
6.827153205871582
6.586665153503418
6.397573471069336
6.348331451416016
5.859395503997803
5.5868144035339355
6.125873565673828
Epoch: [0]  [20/22]  eta: 0:00:16  lr: 0.003000  time: 8.3218  data: 8.1415  max mem: 11177
5.479109764099121
Epoch: [0]  [21/22]  eta: 0:00:08  lr: 0.003000  time: 8.1832  data: 8.0041  max mem: 11177
Epoch: [0] Total time: 0:03:05 (8.4531 s / it)
creating index...
index created!
Test:  [ 0/40]  eta: 0:00:13  model_time: 0.2563 (0.2563)  evaluator_time: 0.0128 (0.0128)  time: 0.3354  data: 0.0626  max mem: 11177
Test:  [39/40]  eta: 0:00:00  model_time: 0.0163 (0.0224)  evaluator_time: 0.0196 (0.0256)

In [ ]:
print('El AP del mejor modelo siendo esre el que guardamos es:')
print(best_ap)
torch.save({
    'model_state_dict': best_model_weights,
}, path_base + "/pesos_modelos/SSD512_ResNet_upgraded_not_augmentation.pth" )

El AP del mejor modelo siendo esre el que guardamos es:
0.23661048252922676


In [ ]:
start_time = time.time()

for epoch in range(50, 100):
    train_one_epoch(model, optimizer, data_loader, device, epoch, 20)
    lr_scheduler.step()
    ap = evaluate(model, data_loader_test, device=device).coco_eval["bbox"].stats[0]
    if ap > best_ap:
      best_ap = ap
      best_model_weights = model.state_dict()

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))

0.12637516856193542
Epoch: [50]  [ 0/22]  eta: 0:00:24  lr: 0.003000  time: 1.0989  data: 0.9177  max mem: 11177
0.1276591420173645
0.09988699853420258
0.09360592067241669
0.08665535598993301
0.17836788296699524
0.08737834542989731
0.11085595190525055
0.11600889265537262
0.15123680233955383
0.14080585539340973
0.1614893525838852
0.10270532965660095
0.11715860664844513
0.15160495042800903
0.1749621480703354
0.14634183049201965
0.114722840487957
0.12190505862236023
0.18964089453220367
0.14015933871269226
Epoch: [50]  [20/22]  eta: 0:00:02  lr: 0.003000  time: 1.0949  data: 0.9242  max mem: 11177
0.11414973437786102
Epoch: [50]  [21/22]  eta: 0:00:01  lr: 0.003000  time: 1.0929  data: 0.9217  max mem: 11177
Epoch: [50] Total time: 0:00:24 (1.0948 s / it)
creating index...
index created!
Test:  [ 0/40]  eta: 0:00:03  model_time: 0.0167 (0.0167)  evaluator_time: 0.0090 (0.0090)  time: 0.0908  data: 0.0614  max mem: 11177
Test:  [39/40]  eta: 0:00:00  model_time: 0.0162 (0.0163)  evaluator_t

In [ ]:
print('El AP del mejor modelo siendo esre el que guardamos es:')
print(best_ap)
torch.save({
    'model_state_dict': best_model_weights,
}, path_base + "/pesos_modelos/SSD512_ResNet_upgraded_not_augmentation.pth" )

El AP del mejor modelo siendo esre el que guardamos es:
0.23661048252922676


In [ ]:
start_time = time.time()


for epoch in range(100, 150):
    train_one_epoch(model, optimizer, data_loader, device, epoch, 20)
    lr_scheduler.step()
    ap = evaluate(model, data_loader_test, device=device).coco_eval["bbox"].stats[0]
    if ap > best_ap:
      best_ap = ap
      best_model_weights = model.state_dict()

total_time = time.time() - start_time
total_time_str = str(datetime.timedelta(seconds=int(total_time)))
print('Training time {}'.format(total_time_str))

0.03025067411363125
Epoch: [100]  [ 0/22]  eta: 0:00:26  lr: 0.000300  time: 1.1975  data: 1.0205  max mem: 11177
0.03687410056591034
0.047358691692352295
0.055377136915922165
0.0448993481695652
0.050554823130369186
0.03261234983801842
0.04032502323389053
0.04570891708135605
0.03762216120958328
0.03840623423457146
0.03668776899576187
0.03735433518886566
0.055757150053977966
0.035263240337371826
0.039823174476623535
0.040758199989795685
0.026283223181962967
0.06008164584636688
0.03794005885720253
0.03769420087337494
Epoch: [100]  [20/22]  eta: 0:00:02  lr: 0.000300  time: 1.0902  data: 0.9197  max mem: 11177
0.03779926151037216
Epoch: [100]  [21/22]  eta: 0:00:01  lr: 0.000300  time: 1.0897  data: 0.9192  max mem: 11177
Epoch: [100] Total time: 0:00:24 (1.0948 s / it)
creating index...
index created!
Test:  [ 0/40]  eta: 0:00:03  model_time: 0.0163 (0.0163)  evaluator_time: 0.0088 (0.0088)  time: 0.0910  data: 0.0621  max mem: 11177
Test:  [39/40]  eta: 0:00:00  model_time: 0.0162 (0.01

In [ ]:
print('El AP del mejor modelo siendo esre el que guardamos es:')
print(best_ap)
torch.save({
    'model_state_dict': best_model_weights,
}, path_base + "/pesos_modelos/SSD512_ResNet_upgraded_not_augmentation.pth" )

El AP del mejor modelo siendo esre el que guardamos es:
0.23661048252922676
